# 🤖 A.B.E.L - Fine-tuning Qwen2.5-7B sur Google Colab

Ce notebook permet de fine-tuner Qwen2.5-7B avec QLoRA sur Google Colab.

**GPU recommandé:**
- T4 (gratuit) - ~15GB VRAM - OK pour batch_size=1
- A100 (Pro) - 40GB VRAM - Optimal

**Temps estimé:** ~2-4 heures selon le dataset

## 1️⃣ Configuration GPU

In [ ]:
# Vérifier le GPU
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2️⃣ Installation des dépendances

In [ ]:
%%capture
!pip install -q torch torchvision torchaudio
!pip install -q transformers>=4.36.0
!pip install -q accelerate>=0.25.0
!pip install -q bitsandbytes>=0.41.0
!pip install -q peft>=0.7.0
!pip install -q trl>=0.7.0
!pip install -q datasets>=2.14.0
!pip install -q wandb

print("✅ Dépendances installées!")

## 3️⃣ Configuration

In [ ]:
# === CONFIGURATION ===

# Modèle
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = "/content/abel-qwen2.5-7b"

# Training
EPOCHS = 3
BATCH_SIZE = 2  # Réduire à 1 si OOM sur T4
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-4
MAX_SEQ_LENGTH = 1024  # Réduire si OOM

# LoRA
LORA_R = 64
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# System prompt A.B.E.L
SYSTEM_PROMPT = """Tu es A.B.E.L (Autonomous Backend Entity for Living), un assistant IA personnel avancé.
Tu es capable de:
- Répondre aux questions de manière précise et concise
- Aider avec la productivité (emails, calendrier, tâches)
- Fournir des informations sur la finance, météo, actualités
- Assister en programmation et développement
- Gérer les divertissements (musique, films, jeux)

Tu réponds en français par défaut. Tu es professionnel, efficace, et tu vas droit au but."""

print("✅ Configuration chargée!")

## 4️⃣ Création du Dataset

In [ ]:
# Dataset d'entraînement A.B.E.L
TRAINING_DATA = [
    # Greetings
    {"user": "Bonjour A.B.E.L!", "assistant": "Bonjour ! Tous mes systèmes sont opérationnels. Comment puis-je vous aider ?"},
    {"user": "Salut!", "assistant": "Salut ! Prêt à vous assister. Que puis-je faire pour vous ?"},
    {"user": "Hey A.B.E.L", "assistant": "Hey ! Je suis à votre écoute."},
    {"user": "Bonsoir", "assistant": "Bonsoir ! Comment puis-je vous être utile ce soir ?"},
    
    # Weather
    {"user": "Quel temps fait-il à Paris?", "assistant": "Paris : 15°C, ciel dégagé. Humidité 65%. Vent 12 km/h. Prévisions : beau temps toute la journée."},
    {"user": "Météo Lyon", "assistant": "Lyon : actuellement 12°C. Min 8°C, Max 16°C. Conditions : nuageux avec éclaircies."},
    {"user": "Il va pleuvoir demain?", "assistant": "Je consulte les prévisions. Pour quelle ville voulez-vous savoir ?"},
    
    # Productivity
    {"user": "Rappelle-moi d'appeler Paul demain", "assistant": "Rappel créé : 'Appeler Paul' pour demain à 09:00. Voulez-vous modifier l'heure ?"},
    {"user": "Crée une tâche: finir le rapport", "assistant": "Tâche ajoutée : 'Finir le rapport'. Priorité : normale. Voulez-vous définir une échéance ?"},
    {"user": "Quels sont mes rendez-vous aujourd'hui?", "assistant": "Vous avez 3 rendez-vous aujourd'hui :\n- 10h00 : Réunion équipe\n- 14h30 : Call client\n- 17h00 : Point projet"},
    {"user": "Résume mes emails non lus", "assistant": "5 emails non lus :\n1. Amazon - Confirmation commande\n2. LinkedIn - Nouvelles connexions\n3. Banque - Relevé disponible\n4. Newsletter Tech - Actualités IA\n5. Contact Pro - Demande de réunion"},
    
    # Finance
    {"user": "Cours du Bitcoin?", "assistant": "Bitcoin (BTC) : 42,350 USD (+2.3% 24h). Volume : 28.5B USD. Capitalisation : 830B USD."},
    {"user": "Ethereum price", "assistant": "Ethereum (ETH) : 2,280 USD (+1.8% 24h). Gas fees : 25 gwei. Staking APY : 4.2%."},
    {"user": "Convertis 100 euros en dollars", "assistant": "100 EUR = 108.50 USD (taux : 1.085). Taux mis à jour il y a 5 minutes."},
    {"user": "Cours de l'action Apple", "assistant": "Apple (AAPL) : 182.50 USD (+0.8%). Volume : 45M. Capitalisation : 2.85T USD."},
    
    # Coding
    {"user": "Écris une fonction Python pour trier une liste", "assistant": "```python\ndef sort_list(lst: list, reverse: bool = False) -> list:\n    return sorted(lst, reverse=reverse)\n```\nUtilisation : `sort_list([3,1,2])` retourne `[1,2,3]`."},
    {"user": "Comment faire une requête HTTP en Python?", "assistant": "```python\nimport requests\n\nresponse = requests.get('https://api.example.com/data')\ndata = response.json()\n```\nInstallez requests avec : `pip install requests`"},
    {"user": "Regex pour valider un email", "assistant": "```python\nimport re\npattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$'\nis_valid = bool(re.match(pattern, email))\n```"},
    {"user": "Différence entre == et is en Python?", "assistant": "`==` compare les valeurs, `is` compare les identités (même objet en mémoire).\n\nExemple :\n```python\na = [1,2]\nb = [1,2]\na == b  # True (mêmes valeurs)\na is b  # False (objets différents)\n```"},
    {"user": "Debug: TypeError list indices", "assistant": "Cette erreur survient quand vous utilisez un mauvais type d'index. Vérifiez que votre variable est bien une liste et non un dict.\n\nExemple d'erreur : `liste['key']` au lieu de `liste[0]`"},
    
    # Entertainment
    {"user": "Films populaires", "assistant": "Top 5 cette semaine :\n1. Dune: Part Two (8.9/10)\n2. Oppenheimer (8.7/10)\n3. Poor Things (8.3/10)\n4. The Holdovers (8.1/10)\n5. Killers of the Flower Moon (8.0/10)"},
    {"user": "Recommande-moi une série", "assistant": "Basée sur les tendances actuelles, je recommande :\n- **True Detective S4** - Thriller\n- **Shogun** - Drame historique\n- **3 Body Problem** - Sci-Fi\n\nQuel genre préférez-vous ?"},
    {"user": "Top anime 2024", "assistant": "Top anime 2024 :\n1. Frieren (9.1)\n2. Solo Leveling (8.8)\n3. Oshi no Ko S2 (8.7)\n4. Jujutsu Kaisen S2 (8.5)"},
    
    # Translation
    {"user": "Traduis 'Hello world' en français", "assistant": "Traduction : 'Bonjour le monde' ou 'Salut le monde' (informel)."},
    {"user": "Comment dit-on 'merci' en japonais?", "assistant": "Merci en japonais :\n- Arigatou (ありがとう) - informel\n- Arigatou gozaimasu (ありがとうございます) - poli"},
    
    # Utilities
    {"user": "Génère un mot de passe", "assistant": "Mot de passe généré : `K9#mP2$xL5@nQ8` (14 caractères, fort). Voulez-vous d'autres options ?"},
    {"user": "Convertis 5 miles en km", "assistant": "5 miles = 8.05 kilomètres."},
    {"user": "Heure à Tokyo?", "assistant": "Tokyo : 18:30 JST (UTC+9). Décalage avec Paris : +8 heures."},
    {"user": "Calcule 15% de 250", "assistant": "15% de 250 = 37.50"},
    
    # About
    {"user": "Qui es-tu?", "assistant": "Je suis A.B.E.L - Autonomous Backend Entity for Living. Un assistant IA personnel conçu pour vous aider dans votre vie quotidienne."},
    {"user": "Que sais-tu faire?", "assistant": "Je peux vous aider avec :\n- Productivité (emails, calendrier, tâches)\n- Finance (crypto, actions, devises)\n- Divertissement (films, musique, jeux)\n- Programmation (code, debug)\n- Utilitaires (météo, traduction, calculs)"},
]

print(f"✅ Dataset créé : {len(TRAINING_DATA)} exemples")

In [ ]:
# Convertir en format Dataset
from datasets import Dataset

def format_conversation(item):
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": item["user"]},
            {"role": "assistant", "content": item["assistant"]},
        ]
    }

formatted_data = [format_conversation(item) for item in TRAINING_DATA]
dataset = Dataset.from_list(formatted_data)

print(f"✅ Dataset formaté : {len(dataset)} exemples")
print(f"\nExemple :")
print(dataset[0])

## 5️⃣ Chargement du Modèle

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Configuration 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

print("📥 Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("📥 Chargement du modèle (4-bit)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = prepare_model_for_kbit_training(model)
print("✅ Modèle chargé!")

In [ ]:
# Appliquer LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 6️⃣ Fine-tuning

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Fonction de formatage
def formatting_func(example):
    return tokenizer.apply_chat_template(example["messages"], tokenize=False)

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    fp16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    report_to="none",  # Désactiver wandb par défaut
    remove_unused_columns=False,
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=formatting_func,
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
)

print("✅ Trainer configuré!")

In [ ]:
# 🚀 LANCER L'ENTRAÎNEMENT
print("="*60)
print("  🚀 DÉBUT DU FINE-TUNING")
print("="*60)

trainer.train()

print("\n" + "="*60)
print("  ✅ FINE-TUNING TERMINÉ!")
print("="*60)

## 7️⃣ Sauvegarde du Modèle

In [ ]:
# Sauvegarder le modèle LoRA
trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Modèle LoRA sauvegardé dans : {OUTPUT_DIR}")

In [ ]:
# Fusionner les poids LoRA
MERGED_DIR = f"{OUTPUT_DIR}-merged"

print("🔄 Fusion des poids LoRA...")
merged_model = model.merge_and_unload()
merged_model.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

print(f"✅ Modèle fusionné sauvegardé dans : {MERGED_DIR}")

## 8️⃣ Test du Modèle

In [ ]:
from transformers import TextStreamer

def chat(user_message, max_tokens=256):
    """Envoie un message au modèle et affiche la réponse."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_message},
    ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    
    print(f"\n👤 USER: {user_message}")
    print("🤖 A.B.E.L: ", end="")
    
    model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        streamer=streamer,
    )
    print()

In [ ]:
# Tests
chat("Bonjour A.B.E.L!")

In [ ]:
chat("Quel temps fait-il à Paris?")

In [ ]:
chat("Écris une fonction Python pour calculer le factoriel")

In [ ]:
chat("Cours du Bitcoin?")

In [ ]:
chat("Qui es-tu et que sais-tu faire?")

## 9️⃣ Télécharger le Modèle

In [ ]:
# Compresser le modèle pour le téléchargement
!zip -r /content/abel-qwen2.5-7b-merged.zip {MERGED_DIR}

print("\n✅ Modèle compressé!")
print("📥 Téléchargez le fichier depuis le panneau de fichiers à gauche")

In [ ]:
# OU uploader sur Hugging Face Hub
# Décommentez et configurez si vous voulez upload sur HF

# from huggingface_hub import login, HfApi
# login(token="YOUR_HF_TOKEN")
# 
# api = HfApi()
# api.upload_folder(
#     folder_path=MERGED_DIR,
#     repo_id="YOUR_USERNAME/abel-qwen2.5-7b",
#     repo_type="model",
# )

## 🎉 Terminé!

Votre modèle A.B.E.L fine-tuné est prêt!

**Options:**
1. Télécharger le zip depuis le panneau de fichiers
2. Upload sur Hugging Face Hub
3. Sauvegarder sur Google Drive

**Utilisation locale:**
```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./abel-qwen2.5-7b-merged")
tokenizer = AutoTokenizer.from_pretrained("./abel-qwen2.5-7b-merged")
```